In [26]:
import csv
import numpy as np

import collections
import operator
from functools import reduce
import json
from multiprocessing import Pool
from keras.preprocessing.sequence import pad_sequences


In [2]:
import xmnlp
xmnlp.radical('花丸晴琉')

(Lazy Load) Loading model...


['艹', '丿', '日', '王']

In [3]:
# Build up Chinese sub-character unites diactionary.
chaiZi_Dic = {}
with open("./corpus/chaizi/chaizi-jt.txt", 'r') as f:
    reader = csv.reader(f,delimiter='\t')
    for row in reader:
        chaiZi_Dic[row[0]] = row[1]
        
def getSubChar(charCN):
    if(charCN in chaiZi_Dic.keys()):
        return chaiZi_Dic[item]
    else:
        return [None]

In [4]:
print(len(chaiZi_Dic.keys()))
for item in "花丸A晴琉":
    print(getSubChar(item))
    print(xmnlp.radical(item))
    print()

14563
草 化
['艹']

丶 九
['丿']

[None]
[None]

日 青
['日']

玉 亠 厶 川
['王']



In [5]:
## Create Corpus for Chinese Word Segamentation

In [6]:
import pandas as pd
pkuTrain = "./corpus/cws/icwb2-data/training/pku_training.utf8"
pkuTest = "./corpus/cws/icwb2-data/testing/pku_test.utf8"
train = pd.read_table(pkuTrain,  encoding='utf8', header=None, names=['input'])  # don't drop the empty lines yet, they show up as NaN in the data frame
train.head(n=1)


,input
0,迈向 充满 希望 的 新 世纪 —— 一九九八年 新年 讲话 （ 附 ...


In [7]:
pkuTest = "./corpus/cws/icwb2-data/testing/pku_test.utf8"
test = pd.read_table(pkuTest,  encoding='utf8', header=None, names=['input']) 
test["raws"] = test["input"]
test.head(n=1)

,input,raws
0,共同创造美好的新世纪——二○○一年新年贺词,共同创造美好的新世纪——二○○一年新年贺词


In [8]:
def prepross_data(train_Data):
    inputs = train_Data["input"][:]
    print(len(inputs))
    raws = []
    tokenList = []
    
    for item in inputs:
        item = item.replace("   ", "  ")

        raw = item.replace(" ", "")
#         print(item)
#         print(raw)
        raws.append(raw)
        tokens = (item.split("  ")[:-1])
        tokenList.append(tokens)
#         print(tokens)
    train_Data["raws"] = raws
    train_Data["tokenList"] = tokenList
    dictionary_train_word = list(set(reduce(operator.add,train_Data["tokenList"])))
    dictionary_train_char =  list(set(reduce(operator.add,train_Data["raws"])))    
    return train_Data, dictionary_train_char, dictionary_train_word
train, train_dic_char, train_dic_word  = prepross_data(train)

19054


In [9]:
# Validate the pre-processing
def validate_data(data):
    tkList = data["tokenList"]
    raws = data["raws"]
    for (tokens, raw) in zip(tkList,raws):
#         print(tokens)
        
#         print(raw)
        temp  = "".join(tokens)
#         print(temp)
        assert(temp == raw)
        if not (temp == raw):
            print(temp)
            print(raw)
            print(tokens)
validate_data(train)

In [10]:
train.head(5)

,input,raws,tokenList
0,迈向 充满 希望 的 新 世纪 —— 一九九八年 新年 讲话 （ 附 ...,迈向充满希望的新世纪——一九九八年新年讲话（附图片１张）,"[迈向, 充满, 希望, 的, 新, 世纪, ——, 一九九八年, 新年, 讲话, （, 附..."
1,中共中央 总书记 、 国家 主席 江 泽民,中共中央总书记、国家主席江泽民,"[中共中央, 总书记, 、, 国家, 主席, 江, 泽民]"
2,（ 一九九七年 十二月 三十一日 ）,（一九九七年十二月三十一日）,"[（, 一九九七年, 十二月, 三十一日, ）]"
3,１２月 ３１日 ， 中共中央 总书记 、 国家 主席 江 泽民 发表 ...,１２月３１日，中共中央总书记、国家主席江泽民发表１９９８年新年讲话《迈向充满希望的新世纪》。...,"[１２月, ３１日, ，, 中共中央, 总书记, 、, 国家, 主席, 江, 泽民, 发表,..."
4,同胞 们 、 朋友 们 、 女士 们 、 先生 们 ：,同胞们、朋友们、女士们、先生们：,"[同胞, 们, 、, 朋友, 们, 、, 女士, 们, 、, 先生, 们, ：]"


In [11]:
print(len(train_dic_char))
print(len(set(train_dic_char)))
print(len(train_dic_word))
print(len(set(train_dic_word)))
with open("./corpus/cws/icwb2-data/gold/pku_training_words.utf8", 'r') as f:
    content = f.readlines()  
    print(len(content))

4698
4698
55303
55303
55303


In [12]:
# Feature extraction

oov = len(train_dic_char)
print(oov)
def token_index(tok):
    ind = tok
    if tok in train_dic_char:  # if token in vocabulary
        ind = train_dic_char.index(tok)
    else:  # else it's OOV
        ind = oov
    return ind

def str_token_index(string):
    return [token_index(x)   for x in string]




# training labels: convert BIO to integers
# def bio_index(bio):
#     ind = bio
#     if not pd.isnull(bio):  # deal with empty lines
#         if bio=='B':
#             ind = 0
#         elif bio=='I':
#             ind = 1
#         elif bio=='O':
#             ind = 2
#     return ind

# Get Begin Middle End Single sequence
# B 0 M 1 E 2 S 3

def str_bmes_idx(tokenList):
    answer = []
    for item in tokenList:
        if len(item) == 0:
            raise NameErro("Zero Length Word")
        if len(item) == 1:
            answer.append(3)
        else:
            answer.append(0)
            for item in range(len(item) - 2):
                answer.append(1)
            answer.append(2)
    return answer




def extract_features(data_set, isTest=False):
    data_temp = data_set.copy()
    

    # Idx for chars
    with Pool(8) as p:
        tokinds = p.map(str_token_index,data_temp['raws'])
#     tokinds = [list(map(token_index, u)) for u in data_temp['raws']]
    data_temp["tokenIdx"] = tokinds
    
    # BIO
    if(not isTest):
        data_temp["bmes"] = [str_bmes_idx(u) for u in data_temp['tokenList']]
        assert (list(map(len,data_temp["bmes"])) == list(map(len,data_temp["tokenIdx"])))

        
#     print(data_temp["bmes"])
#    assert reduce(operator.and_, list(map(len,data_temp["bmes"])) == list(map(len,data_temp["tokenIdx"])))
    
    
    
    
    
    
    return data_temp
#     txt['token_indices'] = tokinds
#     if not istest:  # can't do this with the test set
#         bioints = [bio_index(b) for b in txt['bio_only']]
#         txt['bio_only'] = bioints
#     return txt

4698


In [13]:
%%time
# MultiThreading
temp = extract_features(train)
train_feature = temp
train_feature.head(5)

CPU times: user 435 ms, sys: 44.1 ms, total: 479 ms
Wall time: 15 s


,input,raws,tokenList,tokenIdx,bmes
0,迈向 充满 希望 的 新 世纪 —— 一九九八年 新年 讲话 （ 附 ...,迈向充满希望的新世纪——一九九八年新年讲话（附图片１张）,"[迈向, 充满, 希望, 的, 新, 世纪, ——, 一九九八年, 新年, 讲话, （, 附...","[4670, 1323, 3714, 2382, 4461, 2116, 3018, 544...","[0, 2, 0, 2, 0, 2, 3, 3, 0, 2, 0, 2, 0, 1, 1, ..."
1,中共中央 总书记 、 国家 主席 江 泽民,中共中央总书记、国家主席江泽民,"[中共中央, 总书记, 、, 国家, 主席, 江, 泽民]","[4446, 756, 4446, 522, 3544, 2406, 3659, 2451,...","[0, 1, 1, 2, 0, 1, 2, 3, 0, 2, 0, 2, 3, 0, 2]"
2,（ 一九九七年 十二月 三十一日 ）,（一九九七年十二月三十一日）,"[（, 一九九七年, 十二月, 三十一日, ）]","[3197, 699, 4315, 4315, 1896, 434, 3549, 481, ...","[3, 0, 1, 1, 1, 2, 0, 1, 2, 0, 1, 1, 2, 3]"
3,１２月 ３１日 ， 中共中央 总书记 、 国家 主席 江 泽民 发表 ...,１２月３１日，中共中央总书记、国家主席江泽民发表１９９８年新年讲话《迈向充满希望的新世纪》。...,"[１２月, ３１日, ，, 中共中央, 总书记, 、, 国家, 主席, 江, 泽民, 发表,...","[3440, 2263, 2763, 2996, 3440, 2627, 1528, 444...","[0, 1, 2, 0, 1, 2, 3, 0, 1, 1, 2, 0, 1, 2, 3, ..."
4,同胞 们 、 朋友 们 、 女士 们 、 先生 们 ：,同胞们、朋友们、女士们、先生们：,"[同胞, 们, 、, 朋友, 们, 、, 女士, 们, 、, 先生, 们, ：]","[3281, 4027, 1920, 2451, 2670, 4071, 1920, 245...","[0, 2, 3, 3, 0, 2, 3, 3, 0, 2, 3, 3, 0, 2, 3, 3]"


In [14]:
def find_longest_sequence(data_with_features):
#     assert (np.max(list(map(len, data_with_features["tokenIdx"])))) == (np.max(list(map(len, data_with_features["bmes"]))))
    return (np.max(list(map(len, data_with_features["tokenIdx"]))))
    
    

In [15]:
%%time
temp = extract_features(test, isTest=True)
test_feature = temp
test_feature.head(5)

CPU times: user 18.8 ms, sys: 28 ms, total: 46.8 ms
Wall time: 1.54 s


,input,raws,tokenIdx
0,共同创造美好的新世纪——二○○一年新年贺词,共同创造美好的新世纪——二○○一年新年贺词,"[756, 3281, 1982, 1049, 3284, 610, 3018, 544, ..."
1,（二○○○年十二月三十一日）（附图片1张）,（二○○○年十二月三十一日）（附图片1张）,"[3197, 481, 1352, 1352, 1352, 434, 3549, 481, ..."
2,女士们，先生们，同志们，朋友们：,女士们，先生们，同志们，朋友们：,"[4132, 3009, 1920, 1528, 871, 1090, 1920, 1528..."
3,2001年新年钟声即将敲响。人类社会前进的航船就要驶入21世纪的新航程。中国人民进入了向现代...,2001年新年钟声即将敲响。人类社会前进的航船就要驶入21世纪的新航程。中国人民进入了向现代...,"[174, 3024, 3024, 1379, 434, 544, 434, 4614, 2..."
4,在这个激动人心的时刻，我很高兴通过中国国际广播电台、中央人民广播电台和中央电视台，向全国各族...,在这个激动人心的时刻，我很高兴通过中国国际广播电台、中央人民广播电台和中央电视台，向全国各族...,"[2060, 4658, 3126, 4422, 1988, 3330, 260, 3018..."


In [16]:

train_longest = find_longest_sequence(train_feature)
print(train_longest)

test_longest = find_longest_sequence(test_feature)
print(test_longest)

seq_longest = np.max([train_longest,test_longest])
print(seq_longest)


1019
626
1019


In [28]:
seq_length = seq_longest

# a new dummy token index, one more than OOV
padtok = oov+1
print('The padding token index is %i' % padtok)

# use pad_sequences, padding or truncating at the end of the sequence (default is 'pre')
train_seqs_padded = pad_sequences(train_feature['tokenIdx'].tolist(), maxlen=seq_length,
                                  dtype='int32', padding='post', truncating='post', value=padtok)
print('Example of padded token sequence:')
print(train_seqs_padded[1])

# Prepare Test set.
seq_length = seq_longest
padtok = oov+1

test_seqs_padded = pad_sequences(test_feature['tokenIdx'].tolist(), maxlen=seq_length,
                                  dtype='int32', padding='post', truncating='post', value=padtok)

The padding token index is 4699
Example of padded token sequence:
[4446  756 4446 ... 4699 4699 4699]


In [18]:
from keras.utils import to_categorical

# get lists of named entity labels, padded with a null label (=3)
padlab = 4
train_labs_padded = pad_sequences(train_feature['bmes'].tolist(), maxlen=seq_length,
                                  dtype='int32', padding='post', truncating='post', value=padlab)

# convert those labels to one-hot encoding
n_labs = 5
train_labs_onehot = [to_categorical(i, num_classes=n_labs) for i in train_labs_padded]

# follow the print outputs below to see how the labels are transformed
print('Length of input sequence: %i' % len(train_labs_padded[1]))
print('Length of label sequence: %i' % len(train_labs_onehot[1]))
print(train_labs_padded[1][:11])
print(train_labs_onehot[1][:11])

Length of input sequence: 1019
Length of label sequence: 1019
[0 1 1 2 0 1 2 3 0 2 0]
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]]


Example of padded token sequence:
[3197  481 1352 ... 4699 4699 4699]


In [19]:
# load Keras and TensorFlow
from tensorflow import keras
import tensorflow as tf

# prepare sequences and labels as numpy arrays, check dimensions
X = np.array(train_seqs_padded)
y = np.array(train_labs_onehot)
print('Input sequence dimensions (n.docs, seq.length):')
print(X.shape)
print('Label dimensions (n.docs, seq.length, one-hot encoding of 4 NER labels):')
print(y.shape)

# our final vocab size is the padding token + 1 (OR length of vocab + OOV + PAD)
vocab_size = padtok+1
print(vocab_size==len(train_dic_char)+2)
embed_size = 128  # try an embedding size of 128 (could tune this)

# list of metrics to use: true & false positives, negatives, accuracy, precision, recall, area under the curve
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]

# our model has the option for an label prediction bias, it's sequential, starts with an embedding layer, then bi-LSTM,
# a dropout layer follows for regularisation, and a dense final layer with softmax activation to output class probabilities
# we compile with the Adam optimizer at a low learning rate, use categorical cross-entropy as our loss function
def make_model(metrics = METRICS, output_bias=None):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
    model = keras.Sequential([
        keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_size, input_length=seq_length, mask_zero=True, trainable=True),
        keras.layers.Bidirectional(keras.layers.LSTM(units=50, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)),  # 2 directions, 50 units each, concatenated (can change this)
        keras.layers.Dropout(0.5),
        keras.layers.TimeDistributed(keras.layers.Dense(n_labs, activation='softmax', bias_initializer=output_bias)),
    ])
    model.compile(optimizer=keras.optimizers.Adam(lr=1e-3), loss=keras.losses.CategoricalCrossentropy(), metrics=metrics)
    return model

# early stopping criteria based on area under the curve: will stop if no improvement after 10 epochs
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_auc', verbose=1, patience=10, mode='max', restore_best_weights=True)

# the number of training epochs we'll use, and the batch size (how many texts are input at once)
EPOCHS = 100
BATCH_SIZE = 32

print('**Defining a neural network**')
model = make_model()
model.summary()

Input sequence dimensions (n.docs, seq.length):
(19054, 1019)
Label dimensions (n.docs, seq.length, one-hot encoding of 4 NER labels):
(19054, 1019, 5)
True
**Defining a neural network**
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1019, 128)         601600    
_________________________________________________________________
bidirectional (Bidirectional (None, 1019, 100)         71600     
_________________________________________________________________
dropout (Dropout)            (None, 1019, 100)         0         
_________________________________________________________________
time_distributed (TimeDistri (None, 1019, 5)           505       
Total params: 673,705
Trainable params: 673,705
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.predict(X[:10])

array([[[0.12406749, 0.28121337, 0.13013119, 0.22240639, 0.2421816 ],
        [0.1253646 , 0.27916977, 0.12975103, 0.22159798, 0.24411663],
        [0.1256452 , 0.28069443, 0.12926501, 0.2219209 , 0.24247445],
        ...,
        [0.12096626, 0.28218785, 0.13467443, 0.21780668, 0.24436486],
        [0.1213659 , 0.28169626, 0.13442838, 0.21760549, 0.24490389],
        [0.12198266, 0.28111818, 0.1341656 , 0.21731877, 0.24541482]],

       [[0.12438536, 0.2828231 , 0.13388504, 0.21665809, 0.24224842],
        [0.1251542 , 0.2790904 , 0.13408995, 0.21849519, 0.24317026],
        [0.1246109 , 0.2787367 , 0.13458222, 0.217834  , 0.24423622],
        ...,
        [0.12096626, 0.28218785, 0.13467443, 0.21780668, 0.24436486],
        [0.1213659 , 0.28169626, 0.13442838, 0.2176055 , 0.2449039 ],
        [0.12198266, 0.28111818, 0.1341656 , 0.21731877, 0.24541482]],

       [[0.12728597, 0.28253967, 0.1323207 , 0.21855332, 0.23930039],
        [0.12819177, 0.28037834, 0.1333211 , 0.21740326, 0.2

In [21]:
%%time
# evaluate our initial model
results = model.evaluate(X, y, batch_size=BATCH_SIZE, verbose=0)
print("Loss: {:0.4f}".format(results[0]))

Loss: 1.4584
CPU times: user 37min 29s, sys: 10min 29s, total: 47min 58s
Wall time: 3min 54s


In [22]:
# figure out the label distribution in our fixed-length texts
from collections import Counter

all_labs = [l for lab in train_labs_padded for l in lab]
label_count = Counter(all_labs)
total_labs = len(all_labs)
print(label_count)
print(total_labs)

# use this to define an initial model bias
initial_bias=[(label_count[0]/total_labs), (label_count[1]/total_labs),
              (label_count[2]/total_labs), (label_count[3]/total_labs), (label_count[4]/total_labs)]
print('Initial bias:')
print(initial_bias)

# pass the bias to the model and re-evaluate
model = make_model(output_bias=initial_bias)
results = model.evaluate(X, y, batch_size=BATCH_SIZE, verbose=0)
print("Loss: {:0.4f}".format(results[0]))

Counter({4: 17589578, 0: 585226, 2: 585226, 3: 524721, 1: 131275})
19416026
Initial bias:
[0.03014138938627297, 0.006761167295511451, 0.03014138938627297, 0.027025149224666263, 0.9059309047072763]
Loss: 1.0621


In [25]:
%%time
# re-initiate model with bias
model = make_model(output_bias=initial_bias)

# and fit...
model.fit(X, y, batch_size=BATCH_SIZE, epochs=2, callbacks = [early_stopping],  validation_split = 0.3)

Epoch 1/2
417/417 [==============================] - 470s 1s/step - loss: 0.2245 - tp: 6104143.7871 - fp: 45265.0885 - tn: 182607394.7943 - fn: 39559015.6172 - accuracy: 0.8253 - precision: 0.9838 - recall: 0.1276 - auc: 0.9054 - val_loss: 0.0467 - val_tp: 5704394.0000 - val_fp: 71097.0000 - val_tn: 23231300.0000 - val_fn: 121204.0000 - val_accuracy: 0.9934 - val_precision: 0.9877 - val_recall: 0.9792 - val_auc: 0.9997
Epoch 2/2
417/417 [==============================] - 455s 1s/step - loss: 0.0479 - tp: 6690999.0431 - fp: 84668.0239 - tn: 27240276.9617 - fn: 140236.0478 - accuracy: 0.9933 - precision: 0.9874 - recall: 0.9790 - auc: 0.9996 - val_loss: 0.0383 - val_tp: 5729784.0000 - val_fp: 62606.0000 - val_tn: 23239786.0000 - val_fn: 95814.0000 - val_accuracy: 0.9946 - val_precision: 0.9892 - val_recall: 0.9836 - val_auc: 0.9998
CPU times: user 2h 32min 2s, sys: 35min 12s, total: 3h 7min 14s
Wall time: 15min 24s


In [29]:
X_test = np.array(test_seqs_padded)

 = np.argmax(model.predict(X_test), axis=-1)
flat_preds = [p for pred in preds for p in pred]
print(Counter(flat_preds))

Counter({4: 1808497, 2: 59085, 0: 58638, 3: 44586, 1: 10130})


In [42]:
len(preds)

1944

In [101]:
def postEditPred(pred):
    print(pred)
    ans = []
    for item in pred:
        if item == 0 or item == 3:
            ans.append(1)
        else:
            ans.append(0)
            
    assert len(ans) == len(pred)
    return ans

def splitSentence(st, pred):
    temp_st = st
    temp_pred = pred
    temp_pred[0] = 2
    buf = []
    result = []
    for (pre,char) in zip(temp_pred,temp_st):
        if(pre == 1):
            result.append(buf)
            buf = []
        buf.append(char)
    if(len(buf) > 0):
        result.append(buf)
    return result
    
    



def sentencePrediction(dataset,prediction):
    data_temp = dataset.copy()
    assert len(data_temp) == len(prediction)
    raw_sents = data_temp["raws"]
    assert len(raw_sents) == len(prediction)
    ans = []
    for (st,pred) in zip(raw_sents,prediction):
        sptSt = splitSentence(st,pred)
        ans.append(sptSt)
    assert len(ans) == len(data_temp)
    data_temp["tokenList"]= ans
    return data_temp
    
def convertToPredSts(dataset):
    data_temp = dataset.copy()
    tokenList =   data_temp["tokenList"]
    ans = []
    
    for item in tokenList:
        temp_st = list(map(lambda x : "".join(x), item))
        temp = "  ".join(temp_st)
        ans.append(temp)
    return ans

    
    
result_pred = sentencePrediction(test_feature,preds)
output_sts = convertToPredSts(result_pred)


In [103]:
f = open("predict.txt", "a")
f.write("\n".join(output_sts))
f.close()